In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx

C:\Users\User\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from ema_workbench import load_results
from ema_workbench import (Model, RealParameter, ScalarOutcome, MultiprocessingEvaluator, 
                           ema_logging, Constant, Scenario, Policy, CategoricalParameter, IntegerParameter,
                           perform_experiments, SequentialEvaluator, Constraint)
from ema_workbench.em_framework.evaluators import perform_experiments
from ema_workbench.em_framework.samplers import sample_uncertainties
from ema_workbench.em_framework.optimization import (HyperVolume, EpsilonProgress)
from ema_workbench.util import ema_logging, utilities
import time
from problem_formulation import get_model_for_problem_formulation
#from signal import signal, SIGPIPE, SIG_DFL
#signal(SIGPIPE, SIG_DFL)

ema_logging.log_to_stderr(ema_logging.INFO)

<Logger EMA (DEBUG)>

## MORDM

MORDM consists of four main steps:
1. Problem formulation 
2. Searching for candidate solutions
3. Generating an ensemble of scenarios to explore the effect of uncertainties
4. Using scenario discovery to detect the vulnerabilities of candidate solutions and to improve them


### 1. Problem formulation

For Dike Ring 1, the following objectives, all to be minimized, are of interest:\
•	A.1 Expected Annual Damage\
•	A.1 Dike Investment Costs\
•	A.1 Expected Annual Number of Deaths\
•	Total Expected Annual Number of Deaths

## 2. Searching for candidate solutions

### Constraints

For each outcome of interest we have set a specific constraint that the candidate solutions must follow. We will determine most of them by looking at the random policies that were generated with open exploration.

In [6]:
# function to aggregate over time and locations
def aggregate_df(df):
    df_aggregate = pd.DataFrame()
    locations = ['A.1', 'A.2', 'A.3', 'A.4', 'A.5']
    step = ['0', '1', '2']
    metrics = ['Expected Annual Damage', 'Dike Investment Costs', 'Expected Number of Deaths', 'RfR Total Costs', 'Expected Evacuation Costs']
    
    for metric in metrics:
        if metric == "RfR Total Costs" or metric == "Expected Evacuation Costs":

            columns = [metric + ' ' + time for time in step]

            df_aggregate[metric] = df[columns].sum(axis=1)
        else:
            for location in locations:
                columns = [location + '_' + metric + ' ' + time for time in step]

                df_aggregate[location + '_' + metric] = df[columns].sum(axis=1)
            
            columns_locations = [location + '_' + metric for location in locations]
            df_aggregate['Total ' + metric] = df_aggregate[columns_locations].sum(axis=1)
    
 
    return df_aggregate

In [8]:
outcomes_of_interest = ["A.1_Expected Annual Damage","A.1_Dike Investment Costs",\
                        "A.1_Expected Number of Deaths","Total Expected Number of Deaths",\
                        "Total Expected Annual Damage"]

In [9]:
#Load in the results
results_rp=utilities.load_results('results/policies_fully_disaggregated.tar.gz')
experiments_rp, outcomes_rp = results_rp
df_outcomes_rp_unaggregated = pd.DataFrame(outcomes_rp)

df_rp = aggregate_df(df_outcomes_rp_unaggregated)
df_rp = df_rp[outcomes_of_interest]
df_rp

[MainProcess/INFO] results loaded successfully from C:\Users\User\Documents\GitHub\EPA_Group7_MBDM\final assignment\results\policies_fully_disaggregated.tar.gz


,A.1_Expected Annual Damage,A.1_Dike Investment Costs,A.1_Expected Number of Deaths,Total Expected Number of Deaths,Total Expected Annual Damage
0,0.000000e+00,2.006358e+08,0.000000,0.006540,6.640217e+07
1,0.000000e+00,2.006358e+08,0.000000,0.020768,1.431763e+08
2,0.000000e+00,2.006358e+08,0.000000,0.005515,5.264942e+07
3,0.000000e+00,2.006358e+08,0.000000,0.003705,2.574227e+07
4,0.000000e+00,2.006358e+08,0.000000,0.056785,5.248435e+08
...,...,...,...,...,...
39995,0.000000e+00,1.351227e+08,0.000000,0.000276,1.287416e+06
39996,0.000000e+00,1.351227e+08,0.000000,0.000044,3.102785e+05
39997,0.000000e+00,1.351227e+08,0.000000,0.000039,4.155308e+05
39998,6.111999e+08,1.351227e+08,0.087021,0.087021,6.111999e+08


In [10]:
constraints_dict = {}
constraints_dict[df_rp.columns[1]] = df_rp.iloc[:,1].median()
constraints_dict[df_rp.columns[4]] = df_rp.iloc[:,4].median()
constraints_dict[df_rp.columns[3]] = 0.0001 * 3 * 5

In [11]:
constraints_dict

{'A.1_Dike Investment Costs': 196653651.43629694,
 'Total Expected Annual Damage': 5867396.200260956,
 'Total Expected Number of Deaths': 0.0015}

In [12]:
max_total_deaths = 0.0001 * 3 * 5
max_total_damage = df_rp.iloc[:,4].median()
max_deaths_a1 = df_rp.iloc[:,2].median()
max_damage_a1 = df_rp.iloc[:,0].median()

In [13]:
# Adding the constraint for the allowed maximum number of deaths defined by the Delta Commission. By setting this as a constraint, the optimization will not
# return policies that do not meet this criterion.
constraints = [Constraint("Max Total Number of deaths", outcome_names=['Total Expected Number of Deaths'], function=lambda x:max(0, x-max_total_deaths)),
               #Constraint("Max Total Expected Annual Damage", outcome_names=["Total Expected Annual Damage"], function=lambda x:max(0, x-max_total_damage)),
               #Constraint("Max A.1 Expected Annual Damage", outcome_names=["A.1_Expected Annual Damage"], function=lambda x:max(0, x-max_damage_a1)),
               #Constraint("Max A.1 Expected Number of Deaths", outcome_names=["A.1_Expected Number of Deaths"], function=lambda x:max(0, x-max_deaths_a1))
              ]

In [14]:
outcome_ranges = pd.read_csv("results/outcome_ranges.csv")
outcome_ranges.set_index("Unnamed: 0",inplace=True)
outcome_ranges

,Max_values,Min_values
Unnamed: 0,,
A.1_Expected Annual Damage,3.694513e+09,0.0
A.1_Dike Investment Costs,3.753980e+08,0.0
A.1_Expected Number of Deaths,1.834596e+00,0.0
Total Expected Number of Deaths,5.276779e+00,0.0
Total Expected Annual Damage,5.736794e+09,0.0


In [ ]:
from problem_formulation_modified import get_model_for_problem_formulation_modified
dike_model,  planning_steps = get_model_for_problem_formulation_modified(outcome_ranges)
ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(dike_model) as evaluator:
    results = evaluator.optimize(
        nfe=20,
        searchover="levers",
        epsilons=[
            0.1,0.1,0.1,0.1,0.1
        ]
        * len(dike_model.outcomes),
        constraints=constraints,
    )